# URL 피싱 탐지 모델 학습
## Google Colab용 노트북

**목표:** Random Forest 모델로 악성 URL 탐지 (Accuracy 85%+, Recall 90%+)

**작성자:** 이경준  
**프로젝트:** 스미싱 탐지 시스템 (OCR/QR + URL 위험 판단)

## 1. 환경 설정

In [ ]:
# 필요한 라이브러리 설치
!pip install tldextract python-whois scikit-learn pandas numpy joblib

In [ ]:
import pandas as pd
import numpy as np
import re
from urllib.parse import urlparse
import tldextract
from collections import Counter
import math

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score
)
import joblib

print("✅ 라이브러리 로드 완료")

## 2. 샘플 데이터셋 생성

실제 프로젝트에서는 PhishTank, URLhaus 등에서 데이터를 수집하세요.

In [ ]:
# 악성 URL 샘플
malicious_urls = [
    "http://paypal-secure-login.com/verify",
    "http://amazon-account-verify.com/update",
    "http://apple-id-locked.com/unlock",
    "http://netflix-payment-failed.com/update",
    "http://google-security-alert.com/verify",
    "http://microsoft-account-suspended.com/restore",
    "http://facebook-unusual-activity.com/verify",
    "http://instagram-verify-account.com/confirm",
    "http://cj-delivery-notice.com/parcel",
    "http://hanjin-package-arrived.com/check",
    "http://post-office-notice.com/delivery",
    "http://fedex-customs-clearance.com/pay",
    "http://192.168.1.100/admin/login",
    "http://203.45.67.89/secure/verify",
    "http://10.0.0.1/banking/update",
    "http://bit.ly/free-iphone-giveaway",
    "http://t.co/win-prize-now",
    "http://secure-banking-update-urgent.com",
    "http://verify-your-account-now-2024.com",
    "http://claim-your-reward-immediately.com",
    "http://urgent-action-required-today.com",
    "http://limited-time-offer-act-now.com",
    "http://congratulations-winner-click-here.com",
    "http://account-locked-verify-identity.com",
    "http://suspicious-activity-confirm-now.com",
    "http://pay-pal-secure-login-verify.com",
    "http://face-book-account-recovery.com",
    "http://goo-gle-security-check.com",
    "http://paypa1-secure123.com",
    "http://amaz0n-verify456.com",
]

# 정상 URL 샘플
benign_urls = [
    "https://www.google.com",
    "https://www.youtube.com",
    "https://www.facebook.com",
    "https://www.amazon.com",
    "https://www.wikipedia.org",
    "https://www.twitter.com",
    "https://www.instagram.com",
    "https://www.linkedin.com",
    "https://www.reddit.com",
    "https://www.netflix.com",
    "https://www.naver.com",
    "https://www.daum.net",
    "https://www.kakao.com",
    "https://www.coupang.com",
    "https://www.11st.co.kr",
    "https://www.gmarket.co.kr",
    "https://www.github.com",
    "https://www.stackoverflow.com",
    "https://www.medium.com",
    "https://www.bbc.com",
    "https://www.cnn.com",
    "https://www.nytimes.com",
    "https://www.ebay.com",
    "https://www.walmart.com",
]

# 데이터프레임 생성
df = pd.DataFrame({
    'url': malicious_urls + benign_urls,
    'label': [1] * len(malicious_urls) + [0] * len(benign_urls)
})

# 셔플
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"✅ 데이터셋 생성 완료")
print(f"   - 전체: {len(df)}개")
print(f"   - 악성: {(df['label']==1).sum()}개")
print(f"   - 정상: {(df['label']==0).sum()}개")

df.head()

## 3. 특징 추출 함수 정의

In [ ]:
def extract_features(url):
    """URL에서 30개 특징 추출"""
    features = {}
    
    try:
        parsed = urlparse(url)
        extracted = tldextract.extract(url)
        domain = parsed.netloc
        
        # 1-10: URL 구조
        features['url_length'] = len(url)
        features['domain_length'] = len(domain)
        features['path_length'] = len(parsed.path)
        features['hyphen_count'] = url.count('-')
        features['underscore_count'] = url.count('_')
        features['slash_count'] = url.count('/')
        features['dot_count'] = url.count('.')
        features['has_at_symbol'] = 1 if '@' in url else 0
        features['digit_count'] = sum(c.isdigit() for c in url)
        features['special_char_count'] = len(re.findall(r'[!#$%&*+=?^`{|}~]', url))
        
        # 11-20: 도메인
        features['is_ip_address'] = 1 if re.match(r'^(\d{1,3}\.){3}\d{1,3}$', domain) else 0
        subdomain = extracted.subdomain
        features['subdomain_count'] = len(subdomain.split('.')) if subdomain else 0
        features['domain_has_digits'] = 1 if any(c.isdigit() for c in domain) else 0
        tld = extracted.suffix
        trusted_tlds = ['com', 'org', 'net', 'edu', 'gov']
        features['is_trusted_tld'] = 1 if tld in trusted_tlds else 0
        features['has_www'] = 1 if domain.startswith('www.') else 0
        features['domain_hyphen_count'] = domain.count('-')
        
        # 엔트로피
        if domain:
            char_freq = Counter(domain)
            entropy = 0.0
            for count in char_freq.values():
                prob = count / len(domain)
                entropy -= prob * math.log2(prob)
            features['domain_entropy'] = round(entropy, 4)
        else:
            features['domain_entropy'] = 0.0
        
        features['starts_with_digit'] = 1 if domain and domain[0].isdigit() else 0
        features['domain_dot_count'] = domain.count('.')
        features['is_long_domain'] = 1 if len(domain) > 15 else 0
        
        # 21-30: 콘텐츠
        features['is_https'] = 1 if parsed.scheme == 'https' else 0
        features['has_port'] = 1 if ':' in domain and not domain.startswith('[') else 0
        
        suspicious_keywords = [
            'login', 'signin', 'account', 'update', 'secure', 'banking',
            'verify', 'confirm', 'password', 'urgent', 'suspended', 'locked'
        ]
        url_lower = url.lower()
        features['suspicious_keyword_count'] = sum(1 for kw in suspicious_keywords if kw in url_lower)
        
        query_params = parsed.query.split('&') if parsed.query else []
        features['query_param_count'] = len(query_params)
        features['has_fragment'] = 1 if parsed.fragment else 0
        
        path_depth = len([p for p in parsed.path.split('/') if p])
        features['path_depth'] = path_depth
        
        path = parsed.path
        features['has_file_extension'] = 1 if '.' in path.split('/')[-1] else 0
        features['has_repetitive_chars'] = 1 if re.search(r'(.)\1{2,}', url) else 0
        
        # 브랜드 불일치
        brands = ['paypal', 'amazon', 'google', 'apple', 'microsoft', 'facebook']
        brand_mismatch = 0
        for brand in brands:
            if brand in url_lower and f'{brand}.com' not in url_lower:
                brand_mismatch = 1
                break
        features['brand_mismatch'] = brand_mismatch
        
        short_domains = ['bit.ly', 't.co', 'goo.gl', 'tinyurl.com']
        features['is_shortened_url'] = 1 if any(d in url_lower for d in short_domains) else 0
        
    except Exception as e:
        # 오류 시 기본값
        for i in range(30):
            if f'feature_{i}' not in features:
                features[f'feature_{i}'] = 0
    
    return features

print("✅ 특징 추출 함수 정의 완료")

## 4. 전체 데이터셋에 특징 추출 적용

In [ ]:
print("🔍 특징 추출 시작...")

features_list = []
for idx, url in enumerate(df['url']):
    features = extract_features(url)
    features_list.append(features)
    
    if (idx + 1) % 10 == 0:
        print(f"   진행: {idx+1}/{len(df)}")

# DataFrame으로 변환
X = pd.DataFrame(features_list)
y = df['label']

print(f"\n✅ 특징 추출 완료")
print(f"   - 샘플 수: {len(X)}")
print(f"   - 특징 수: {len(X.columns)}")

X.head()

## 5. 데이터 분할

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"✅ 데이터 분할 완료")
print(f"   - Train: {len(X_train)}개")
print(f"   - Test:  {len(X_test)}개")

## 6. 모델 학습

In [ ]:
print("🤖 Random Forest 모델 학습 시작...\n")

model = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

model.fit(X_train, y_train)

print("\n✅ 모델 학습 완료")

## 7. 모델 평가

In [ ]:
# 예측
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Train 성능
print("[Train 성능]")
print(f"Accuracy: {accuracy_score(y_train, y_pred_train):.4f}")

# Test 성능
print("\n[Test 성능]")
acc = accuracy_score(y_test, y_pred_test)
prec = precision_score(y_test, y_pred_test)
rec = recall_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)

print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1-Score:  {f1:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_test)
print("\n[Confusion Matrix]")
print(f"TN: {cm[0][0]}  FP: {cm[0][1]}")
print(f"FN: {cm[1][0]}  TP: {cm[1][1]}")

# Classification Report
print("\n[Classification Report]")
print(classification_report(y_test, y_pred_test, target_names=['Benign', 'Malicious']))

# 목표 달성
print("\n🎯 목표 달성 여부:")
print(f"   - Accuracy ≥ 85%: {'✅' if acc >= 0.85 else '❌'} ({acc:.1%})")
print(f"   - Recall ≥ 90%:   {'✅' if rec >= 0.90 else '❌'} ({rec:.1%})")

## 8. 특징 중요도

In [ ]:
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("[상위 10개 중요 특징]")
print(feature_importance.head(10))

# 시각화 (선택)
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.barh(feature_importance.head(10)['feature'], feature_importance.head(10)['importance'])
plt.xlabel('Importance')
plt.title('Top 10 Feature Importance')
plt.gca().invert_yaxis()
plt.show()

## 9. 모델 저장

In [ ]:
# 모델 저장
joblib.dump(model, 'url_classifier.pkl')
print("✅ 모델 저장: url_classifier.pkl")

# 특징 이름 저장
joblib.dump(list(X.columns), 'feature_names.pkl')
print("✅ 특징 이름 저장: feature_names.pkl")

# 메타데이터 저장
metadata = {
    'model_type': 'RandomForest',
    'n_features': len(X.columns),
    'accuracy': acc,
    'precision': prec,
    'recall': rec,
    'f1_score': f1
}
joblib.dump(metadata, 'metadata.pkl')
print("✅ 메타데이터 저장: metadata.pkl")

print("\n📥 다운로드할 파일:")
print("   1. url_classifier.pkl")
print("   2. feature_names.pkl")
print("   3. metadata.pkl")

## 10. 테스트 (새로운 URL 예측)

In [ ]:
def predict_url(url):
    """새 URL의 위험도 예측"""
    features = extract_features(url)
    features_df = pd.DataFrame([features])
    
    prediction = model.predict(features_df)[0]
    proba = model.predict_proba(features_df)[0]
    
    result = "악성" if prediction == 1 else "정상"
    confidence = proba[prediction] * 100
    
    print(f"URL: {url}")
    print(f"예측: {result} (신뢰도: {confidence:.1f}%)")
    print(f"확률: 정상 {proba[0]:.2%} | 악성 {proba[1]:.2%}")
    return prediction

# 테스트
test_urls = [
    "https://www.google.com",
    "http://paypal-secure.com/verify",
    "http://192.168.1.1/admin"
]

for url in test_urls:
    print("\n" + "="*60)
    predict_url(url)